In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

import datas.mnist_data as mnist

In [1]:
# 학습 데이터(MNIST)
(train_labels, train_images) = mnist.get_data('./datas/', 'train')

# X(이미지)와 Y(숫자값)의 입력값
X = tf.placeholder(tf.float32, [None, 28*28]) # 무한대 x 784(28*28) 행렬
Y = tf.placeholder(tf.int64, [None]) # 무한대 x 1 행렬

In [2]:
# CNN Model
def model(input_X):
    x_reshape = tf.reshape(X, [-1, 28, 28, 1])

    # Conv 레이어1
    W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 20], stddev=0.1)) # Filter가 Weight 역활을 함
    b_conv1 = tf.Variable(tf.zeros([20]))
    h_conv1 = tf.nn.conv2d(x_reshape, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
    fmap_conv1 = tf.nn.tanh(h_conv1) # -> Feature(Activation) Map 생성

    # Pooling(Max) 레이어1
    h_pool1 = tf.nn.max_pool(fmap_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Conv 레이어2
    W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 20, 50], stddev=0.1)) # Filter가 Weight 역활을 함
    b_conv2 = tf.Variable(tf.zeros([50]))
    h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2
    fmap_conv2 = tf.nn.tanh(h_conv2) # -> Feature(Activation) Map 생성

    # Pooling(Max) 레이어2
    h_pool2 = tf.nn.max_pool(fmap_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # fully-connected 레이어 1
    W_fc1 = tf.Variable(tf.truncated_normal([50 * 7 * 7, 500], stddev=0.1))
    b_fc1 = tf.Variable(tf.zeros([500]))
    h_pool2_flat = tf.reshape(h_pool2, [-1, 50 * 7 * 7])
    h_fc1 = tf.nn.tanh(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    # 출력(fully connected) 레이어 2 (10개 출력)
    class_num = 10
    W_fc2 = tf.Variable(tf.truncated_normal([500, class_num], stddev=0.1))
    b_fc2 = tf.Variable(tf.zeros([class_num]))
    pred = tf.matmul(h_fc1, W_fc2) + b_fc2

    return pred

pred = model(X)

In [3]:
# Cost Function 설계 (Cross Entropy)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=pred)
cost = tf.reduce_mean(cross_entropy)

# Gradient descent Optimizer(학습)
# 미분을 통해서 해당 점의 기울기가 가장 작은 곳이 최적화의 포인트(learning_rate만큼의 단위로 실행)
# 지속적으로 기울기(미분)를 측정하여 W와 b를 수정
# W' = W - (cost함수의 미분값 * learning_rate:0.01)
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [4]:
# 학습 시작
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    index_in_epoch = 0
    # 학습횟수(epoch:60) -> 총 60000개
    for epoch in range(1, 60):
        start = index_in_epoch
        index_in_epoch += 1000 # 배치 1000개
        end = index_in_epoch

        X_images = np.reshape(train_images[start:end], [-1, 28*28])
        Y_labels = train_labels[start:end]

        sess.run(optimizer, feed_dict={X: X_images , Y: Y_labels})

        # 로그
        training_cost = sess.run(cost, feed_dict={X: X_images , Y: Y_labels})
        print(epoch, training_cost)
        #print(epoch, training_cost, [W.eval(), b.eval()])

    print("학습완료! (cost : " + str(training_cost) + ")")

    # 테스트 데이터로 테스트(총 10000개)
    # iteration없고 optimizer없이, 테스트 데이터만 가지고 체크
    # => cost 안에 이미 W와 b가 결정되었기 때문
    (test_labels, test_images) = mnist.get_data('./datas/', 'test')

    test_X = np.reshape(test_images, [-1, 28*28])
    test_Y = test_labels

    testing_cost = sess.run(cost, feed_dict={X: test_X, Y: test_Y})
    print("테스트 완료! (cost : " + str(testing_cost) + ")")

    # 학습과 테스트 cost비교(절대값)
    print("테스트와 학습의 cost차이 : ", abs(training_cost - testing_cost))
    
    # 값 예측 (10개)
    for i in range(20):
        x_test = np.reshape(train_images[i], [-1, 28*28])
        arr_data = sess.run(pred, feed_dict={X: x_test})

        pred_val = tf.argmax(arr_data, 1)
        real_val = train_labels[i]

        print("예측값: " + str(pred_val.eval()) + " / 실제값" + str(real_val) + " => " + str(tf.equal(pred_val, real_val).eval()))

1 2.8834023
2 2.6790378
3 2.4239645
4 2.3403795
5 2.2267876
6 2.088479
7 2.0201745
8 2.0252404
9 1.8870864
10 1.8430514
11 1.776801
12 1.7145612
13 1.7280375
14 1.6976445
15 1.7082736
16 1.5753155
17 1.5494329
18 1.6005075
19 1.3957056
20 1.425155
21 1.4269668
22 1.3697368
23 1.4294217
24 1.3080713
25 1.3055706
26 1.226788
27 1.2051196
28 1.2498722
29 1.1519587
30 1.2208596
31 1.1905044
32 1.2271297
33 1.2298484
34 1.0491599
35 1.0995607
36 1.0753398
37 0.9874339
38 1.0684307
39 0.9832345
40 1.0282269
41 1.0153321
42 1.0206051
43 1.0364976
44 0.9581321
45 1.0002047
46 0.97041166
47 0.93552005
48 0.9452676
49 0.8229323
50 1.0153134
51 0.8518617
52 0.8388412
53 0.85759705
54 0.8223206
55 0.8395384
56 0.7701665
57 0.7826421
58 0.80220175
59 0.6887384
학습완료! (cost : 0.6887384)
테스트 완료! (cost : 0.8109547)
테스트와 학습의 cost차이 :  0.122216284
예측값: [3] / 실제값5 => [False]
예측값: [0] / 실제값0 => [ True]
예측값: [4] / 실제값4 => [ True]
예측값: [1] / 실제값1 => [ True]
예측값: [9] / 실제값9 => [ True]
예측값: [2] / 실제값2 => [ Tru